# Pulling Data from the ODP API
Now the data is stored in ODP, we can test to see if its working and start to think about how we might analyse it. To make things a bit more managable, you might want to filter the data by county and year so lets make a something to help with that. We then use this input to filter the data we pull from the API.

In [36]:
import requests
import json
import geojson
import ipywidgets as widgets

import pandas as pd
import geopandas as gpd
import xarray as xr

from shapely.geometry import Point
import pydeck as pdk

In [37]:
token = requests.post("http://localhost:8000/access_token").json()['token']

In [38]:
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
    "X-ODP-CHUNKED-ENCODING": "false"
}
base_url = "https://api.hubocean.earth"
dataset_name = "overview-of-the-live-stock-1998-2022_tr" # Use the existing name or make-your-own-computer-friendly-name

In [39]:
# Define the list of counties
county_list = [
    'Troms og Finnmark',
    'Nordland',
    'Trøndelag',
    'Møre og Romsdal',
    'Vestland',
    'Rogaland',
    'Øvrige fylker'
]

# Define a dropdown widget for selecting the county
county_dropdown = widgets.Dropdown(
    options=county_list,
    value=county_list[0],  # Set the default selected county
    description='Select County:',
    disabled=False,
)

# Create a slider widget
year_slider = widgets.IntSlider(
    value=2022,  # Initial value
    min=1998,    # Minimum year
    max=2030,    # Maximum year
    step=1,      # Step size
    description='Select Year:',
    continuous_update=False  # Set to False to update only on slider release
)

In [40]:
def vizualization():
    INITIAL_VIEW_STATE = pdk.ViewState(latitude=65, longitude=6, zoom=3, max_zoom=16, pitch=30, bearing=0)

    layer = pdk.Layer(
        "GridCellLayer",
        data=df,
        get_position="[Location.coordinates[1], Location.coordinates[0]]",
        get_elevation="Losses",
        get_color="[255, 0, 0]",
        cellSize=50000,
        coverage=1,
        opacity=1,
        extruded=True,
        material=False,
        pickable=True,
        elevationScale=7
    )

    latitude_shift = 0
    longitude_shift = -1

    layer2 = pdk.Layer(
        "GridCellLayer",
        data=df,
        get_position=f"[Location.coordinates[1] + {longitude_shift}, Location.coordinates[0] + {latitude_shift}]",
        get_elevation="Output",
        get_color="[0, 255, 0]",
        cellSize=50000,
        coverage=1,
        opacity=1,
        extruded=True,
        material=False,
        pickable=True,
        elevationScale=7
    )

    r = pdk.Deck(layers=[layer2, layer],
                api_keys={'mapbox':'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'},
                map_provider='mapbox',
                map_style='mapbox://styles/oceandatafoundation/ckbujyuqu0kah1kpbctq4lemg',
                initial_view_state=INITIAL_VIEW_STATE,
                )
    return r

In [41]:
def data_API_call(filter):
# Query for our dataset with the OQS syntax.
    resource_group = "catalog.hubocean.io"
    resource_type = "dataset"
    endpoint = f"/data/{resource_group}/{resource_type}/{dataset_name}/list"
    url = base_url + endpoint
    if filter == "County":
        body = {
                    "#EQUALS": ["$County", county_dropdown.value]
        }
    else:
        body = {
                    "#EQUALS": ["$Year", year_slider.value]
        }
    response = requests.post(url, json=body, headers=headers)
    if response.status_code == 200:
        json_response = response.json()
        return pd.json_normalize(json_response['data'], max_level=0)
    else:
        return print(f"Request failed with status code {response.status_code} - {response.text}")


In [42]:
display(year_slider)

IntSlider(value=2022, continuous_update=False, description='Select Year:', max=2030, min=1998)

In [45]:
df = data_API_call("Year")
vizualization()

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 65,
    "longitude": 6,
    "maxZoom": 16,
    "pitch": 30,
    "zoom": 3
  },
  "layers": [
    {
      "@@type": "GridCellLayer",
      "cellSize": 50000,
      "coverage": 1,
      "data": [
        {
          "County": "M\u00f8re og Romsdal",
          "Input": 16656.463,
          "Live stock pr. 1.1": 21785.006,
          "Live stock pr. 12.31.": 21484.204,
          "Location": {
            "coordinates": [
              62.9849,
              6.5318
            ],
            "type": "Point"
          },
          "Losses": 3650.216,
          "Output": 13307.049,
          "Species": "Atlantic Salmon",
          "Year": 2008
        },
        {
          "County": "M\u00f8re og Romsdal",
          "Input": 4114.788,
          "Live stock pr. 1.1": 6569.893,
          "Live stock pr. 12.31.": 4981.504,
          "Location": {
            "coordinates": [
              62.9849,
              6.5318
            ],
            "type": "Point"
          },
          "Losses": 773.242,
          "Output": 4929.935,
          "Species": "Rainbow Trout",
          "Year": 2008
        },
        {
          "County": "Nordland",
          "Input": 14952.186,
          "Live stock pr. 1.1": 48410.434,
          "Live stock pr. 12.31.": 21783.144,
          "Location": {
            "coordinates": [
              67.419,
              12.918
            ],
            "type": "Point"
          },
          "Losses": 5260.744,
          "Output": 36318.732,
          "Species": "Atlantic Salmon",
          "Year": 2008
        },
        {
          "County": "Nordland",
          "Input": 2232.258,
          "Live stock pr. 1.1": 3770.357,
          "Live stock pr. 12.31.": 2733.498,
          "Location": {
            "coordinates": [
              67.419,
              12.918
            ],
            "type": "Point"
          },
          "Losses": 352.798,
          "Output": 2916.319,
          "Species": "Rainbow Trout",
          "Year": 2008
        },
        {
          "County": "Rogaland",
          "Input": 23461.947,
          "Live stock pr. 1.1": 33604.634,
          "Live stock pr. 12.31.": 32751.381,
          "Location": {
            "coordinates": [
              58.7555,
              5.2953
            ],
            "type": "Point"
          },
          "Losses": 5072.927,
          "Output": 19242.273,
          "Species": "Atlantic Salmon",
          "Year": 2008
        },
        {
          "County": "Rogaland",
          "Input": 2243.682,
          "Live stock pr. 1.1": 3330.558,
          "Live stock pr. 12.31.": 3013.881,
          "Location": {
            "coordinates": [
              58.7555,
              5.2953
            ],
            "type": "Point"
          },
          "Losses": 159.603,
          "Output": 2400.756,
          "Species": "Rainbow Trout",
          "Year": 2008
        },
        {
          "County": "\u00d8vrige fylker",
          "Input": 77452.0,
          "Live stock pr. 1.1": 94307.0,
          "Live stock pr. 12.31.": 93755.0,
          "Location": {
            "coordinates": [
              61.406,
              8.631
            ],
            "type": "Point"
          },
          "Losses": 9101.0,
          "Output": 68904.0,
          "Species": "Atlantic Salmon",
          "Year": 2008
        },
        {
          "County": "\u00d8vrige fylker",
          "Input": 0.0,
          "Live stock pr. 1.1": 741.0,
          "Live stock pr. 12.31.": 0.0,
          "Location": {
            "coordinates": [
              61.406,
              8.631
            ],
            "type": "Point"
          },
          "Losses": 142.0,
          "Output": 599.0,
          "Species": "Rainbow Trout",
          "Year": 2008
        },
        {
          "County": "M\u00f8re og Romsdal",
          "Input": 18785.162,
          "Live stock pr. 1.1": 21372.514,
          "Live stock pr. 12.31.": 24289.118,

In [46]:
# Display the dropdown widget
display(county_dropdown)

Dropdown(description='Select County:', options=('Troms og Finnmark', 'Nordland', 'Trøndelag', 'Møre og Romsdal…

In [50]:
data_API_call("County").head(140).tail(19)

,Live stock pr. 12.31.,Output,Input,Year,Losses,Live stock pr. 1.1,County,Species,Location
121,2733.498,2916.319,2232.258,2008,352.798,3770.357,Nordland,Rainbow Trout,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
122,46129.054,18421.592,48116.168,2009,5452.014,21886.492,Nordland,Atlantic Salmon,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
123,2742.958,2931.326,3191.707,2009,250.921,2733.498,Nordland,Rainbow Trout,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
124,46129.054,18421.592,48116.168,2010,5452.014,21886.492,Nordland,Atlantic Salmon,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
125,2742.958,2931.326,3191.707,2010,250.921,2733.498,Nordland,Rainbow Trout,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
126,19739.111,32951.977,13065.155,2011,6400.120,46026.053,Nordland,Atlantic Salmon,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
127,3757.110,2192.186,3522.648,2011,316.310,2742.958,Nordland,Rainbow Trout,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
128,19739.111,32951.977,13065.155,2012,6400.120,46026.053,Nordland,Atlantic Salmon,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
129,3757.110,2192.186,3522.648,2012,316.310,2742.958,Nordland,Rainbow Trout,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
130,51902.617,16156.600,54031.379,2013,5526.127,19553.965,Nordland,Atlantic Salmon,"{'type': 'Point', 'coordinates': [67.419, 12.9..."
